# Airline delays 
## Bureau of Transportation Statistics
https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp   
https://www.bts.gov/topics/airlines-and-airports/understanding-reporting-causes-flight-delays-and-cancellations

~140GB

In [2]:
# imports
import re
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import os
from pyspark.sql.types import IntegerType, TimestampType
import copy
import pyspark.sql.functions as F
from functools import reduce
from pyspark.sql import DataFrame



#%matplotlib inline

In [3]:
dbutils.fs.ls('/databricks-datasets/airlines')

Out[2]: [FileInfo(path='dbfs:/databricks-datasets/airlines/README.md', name='README.md', size=1089),
 FileInfo(path='dbfs:/databricks-datasets/airlines/_SUCCESS', name='_SUCCESS', size=0),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00000', name='part-00000', size=67108879),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00001', name='part-00001', size=67108862),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00002', name='part-00002', size=67108930),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00003', name='part-00003', size=67108804),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00004', name='part-00004', size=67108908),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00005', name='part-00005', size=67108890),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00006', name='part-00006', size=67108825),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00007', name='part-00007', size=67108880),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00008', name='part-00008', size=67108832),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00009', name='part-00009', size=67108857),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00010', name='part-00010', size=67108931),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00011', name='part-00011', size=67108872),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00012', name='part-00012', size=67108791),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00013', name='part-00013', size=67108856),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00014', name='part-00014', size=67108855),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00015', name='part-00015', size=67108875),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00016', name='part-00016', size=67108898),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00017', name='part-00017', size=67108873),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00018', name='part-00018', size=67108881),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00019', name='part-00019', size=67108829),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00020', name='part-00020', size=67108863),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00021', name='part-00021', size=67108868),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00022', name='part-00022', size=67108827),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00023', name='part-00023', size=67108926),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00024', name='part-00024', size=67108885),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00025', name='part-00025', size=67108843),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00026', name='part-00026', size=67108837),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00027', name='part-00027', size=67108911),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00028', name='part-00028', size=67108776),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00029', name='part-00029', size=67108929),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00030', name='part-00030', size=67108839),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00031', name='part-00031', size=67108867),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00032', name='part-00032', size=67108865),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00033', name='part-00033', size=67108864),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00034', name='part-00034', size=67108854),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00035', name='part-00035', size=67108878),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00036', name='part-00036', size=67108847),
 FileInfo(path='dbfs:/databricks-datasets/airlines/part-00037', name='part-00037', size=67108895),
 FileInfo(path='dbfs:/databricks-datasets/airline

In [4]:
sum = 0
DATA_PATH = 'dbfs:/databricks-datasets/airlines/'
for item in dbutils.fs.ls(DATA_PATH):
  sum = sum+item.size
sum


Out[3]: 128911081815

In [5]:
with open("/dbfs/databricks-datasets/airlines/README.md") as f:
    x = ''.join(f.readlines())

print(x)

================================================
Airline On-Time Statistics and Delay Causes
================================================

## Background
The U.S. Department of Transportation's (DOT) Bureau of Transportation Statistics (BTS) tracks the on-time performance of domestic flights operated by large air carriers. Summary information on the number of on-time, delayed, canceled and diverted flights appears in DOT's monthly Air Travel Consumer Report, published about 30 days after the month's end, as well as in summary tables posted on this website. BTS began collecting details on the causes of flight delays in June 2003. Summary statistics and raw data are made available to the public at the time the Air Travel Consumer Report is released.


FAQ Information is available at http://www.rita.dot.gov/bts/help_with_data/aviation/index.html


## Data Source
http://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp


## Usage Restrictions
The data is released under the Freedom of Information act. More information can be found at http://www.fas.org/sgp/foia/citizen.html

In [6]:
#union the files.
names = ["dbfs:/databricks-datasets/airlines/part-{:05d}".format(i) for i in range(10)]

all_airlines = [spark.read.option("header", "true").csv(names) for name in names]

airlines = reduce(DataFrame.unionAll, all_airlines)

# airlines = spark.read.option("header", "true").csv("dbfs:/databricks-datasets/airlines/part-00000")

# airlinesOthers = spark.read.option("header", "true").csv("dbfs:/databricks-datasets/airlines/part-00997")

/databricks/spark/python/pyspark/sql/dataframe.py:1500: DeprecationWarning: Deprecated in 2.0, use union instead.
 warnings.warn("Deprecated in 2.0, use union instead.", DeprecationWarning)

In [7]:
display(airlines.take(100))
#display(airlinesOthers.take(100))

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
1987,10,14,3,741,730,912,849,PS,1451,NA,91,79,NA,23,11,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
1987,10,15,4,729,730,903,849,PS,1451,NA,94,79,NA,14,-1,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,NO
1987,10,17,6,741,730,918,849,PS,1451,NA,97,79,NA,29,11,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
1987,10,18,7,729,730,847,849,PS,1451,NA,78,79,NA,-2,-1,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,NO,NO
1987,10,19,1,749,730,922,849,PS,1451,NA,93,79,NA,33,19,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
1987,10,21,3,728,730,848,849,PS,1451,NA,80,79,NA,-1,-2,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,NO,NO
1987,10,22,4,728,730,852,849,PS,1451,NA,84,79,NA,3,-2,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,NO
1987,10,23,5,731,730,902,849,PS,1451,NA,91,79,NA,13,1,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
1987,10,24,6,744,730,908,849,PS,1451,NA,84,79,NA,19,14,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,YES
1987,10,25,7,729,730,851,849,PS,1451,NA,82,79,NA,2,-1,SAN,SFO,447,NA,NA,0,NA,0,NA,NA,NA,NA,NA,YES,NO


In [8]:
sc = spark.sparkContext
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 spark://10.74.242.103:7077 
 AppName 
 Databricks Shell

In [9]:
dir(airlines)
airlines.__class__

Out[8]: pyspark.sql.dataframe.DataFrame

### About the Data:
* ... __Categorical:__ Year, Month, DayofMonth, DayofWeek, UniqueCarrier, FlightNumber, Origin, Dest
* ... __Continuous:__ Distance

* ... __Input Time:__ CRSDepTime, CRSArrTime, CRSElapsedTime

* ...__Targets:__ ArrDelay, DepDelay, IsArrDelayed,IsDepDealyed  (use the last two variables for accuracy).  The other approach is to use the Output Time variables
* ...__Output Time:__ DepTime, ArrTime, ActualElapsedTime 

* ...__Trash:__ TailNumber, Air Time, TaxiIn, TaxiOut, CancellationCode
* ...__Other Output:__ Cancelled, Diverted, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay 

* ..."DataType", "NullType", "StringType", "BinaryType", "BooleanType", "DateType","TimestampType", "DecimalType", "DoubleType", "FloatType", "ByteType", "IntegerType","LongType", "ShortType", "ArrayType", "MapType", "StructField", "StructType"]

In [11]:
airlines.schema.names

Out[9]: ['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'CRSDepTime',
 'ArrTime',
 'CRSArrTime',
 'UniqueCarrier',
 'FlightNum',
 'TailNum',
 'ActualElapsedTime',
 'CRSElapsedTime',
 'AirTime',
 'ArrDelay',
 'DepDelay',
 'Origin',
 'Dest',
 'Distance',
 'TaxiIn',
 'TaxiOut',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay',
 'IsArrDelayed',
 'IsDepDelayed']

In [12]:
#  Data fixing

timeVariables = ['CRSDepTime', 'CRSArrTime', 'CRSElapsedTime','DepTime','ArrTime']
intVariables = ['Distance','ArrDelay','DepDelay','ActualElapsedTime']
deletedVariables = ['TailNum', 'AirTime', 'TaxiIn', 'TaxiOut', 'CancellationCode','Cancelled', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay','FlightNum']
#Take FlightNumber out of the list row 39 issue, will come back later time
catagoricalVaribles = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Origin', 'Dest']


# for var in timeVariables:
#   airlines = airlines.withColumn(var, airlines[var].cast(TimestampType()))

for var in intVariables:
  airlines = airlines.withColumn(var, airlines[var].cast(IntegerType()))

for var in deletedVariables:
  airlines = airlines.drop(var)

for var in catagoricalVaribles:
  other_cols = airlines.schema.names
#   print(other_cols)
  print(var)
  
  other_cols.remove(var)
  codes = airlines.select(var).distinct().rdd.flatMap(lambda x: x).collect()
  print(codes)
  codes_expr = [F.when(F.col(var) == code, 1).otherwise(0).alias(var +'_'+ code) for code in codes]
  airlines = airlines.select(*other_cols,*codes_expr)
  

display(airlines.take(100))
#   df = df.select("ID", "TYPE", "CODE", *types_expr+codes_expr)
#   df.show()



DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,ArrDelay,DepDelay,Distance,IsArrDelayed,IsDepDelayed,Year_1987,Year_1988,Month_7,Month_11,Month_3,Month_8,Month_5,Month_6,Month_9,Month_1,Month_10,Month_4,Month_12,Month_2,DayofMonth_7,DayofMonth_15,DayofMonth_11,DayofMonth_29,DayofMonth_3,DayofMonth_30,DayofMonth_8,DayofMonth_22,DayofMonth_28,DayofMonth_16,DayofMonth_5,DayofMonth_31,DayofMonth_18,DayofMonth_27,DayofMonth_17,DayofMonth_26,DayofMonth_6,DayofMonth_19,DayofMonth_23,DayofMonth_25,DayofMonth_24,DayofMonth_9,DayofMonth_1,DayofMonth_20,DayofMonth_10,DayofMonth_4,DayofMonth_12,DayofMonth_13,DayofMonth_21,DayofMonth_14,DayofMonth_2,DayOfWeek_7,DayOfWeek_3,DayOfWeek_5,DayOfWeek_6,DayOfWeek_1,DayOfWeek_4,DayOfWeek_2,UniqueCarrier_UA,UniqueCarrier_EA,UniqueCarrier_PI,UniqueCarrier_PS,UniqueCarrier_AA,UniqueCarrier_NW,UniqueCarrier_HP,UniqueCarrier_TW,UniqueCarrier_DL,UniqueCarrier_US,UniqueCarrier_AS,UniqueCarrier_CO,UniqueCarrier_PA (1),UniqueCarrier_WN,Origin_BGM,Origin_MSY,Origin_GEG,Origin_BUR,Origin_SNA,Origin_GTF,Origin_GRB,Origin_IDA,Origin_GRR,Origin_EUG,Origin_PSG,Origin_MYR,Origin_GSO,Origin_PVD,Origin_ISO,Origin_OAK,Origin_MSN,Origin_FAR,Origin_BTM,Origin_SCC,Origin_DET,Origin_DCA,Origin_CID,Origin_MLU,Origin_WRG,Origin_HLN,Origin_LEX,Origin_RDM,Origin_ORF,Origin_EVV,Origin_KTN,Origin_CRW,Origin_CWA,Origin_SAV,Origin_TRI,Origin_CDV,Origin_CMH,Origin_SPN,Origin_UCA,Origin_CAK,Origin_CHO,Origin_MOB,Origin_PNS,Origin_LIH,Origin_IAH,Origin_HNL,Origin_ERI,Origin_SHV,Origin_GST,Origin_CVG,Origin_SJC,Origin_TOL,Origin_LGA,Origin_BUF,Origin_TLH,Origin_HPN,Origin_RDD,Origin_AUS,Origin_MLI,Origin_SJU,Origin_ATW,Origin_LYH,Origin_AVL,Origin_GJT,Origin_LGB,Origin_BFL,Origin_GFK,Origin_RNO,Origin_SRQ,Origin_EYW,Origin_SBN,Origin_JAC,Origin_RST,Origin_CHS,Origin_RSW,Origin_TUL,Origin_HRL,Origin_AMA,Origin_ISP,Origin_BOS,Origin_MAF,Origin_MLB,Origin_EWR,Origin_LAS,Origin_BIS,Origin_FAI,Origin_JAN,Origin_DLH,Origin_DEN,Origin_ALB,Origin_CPR,Origin_LNK,Origin_OME,Origin_IAD,Origin_PSP,Origin_SBA,Origin_BOI,Origin_DRO,Origin_SEA,Origin_CMI,Origin_LAN,Origin_MCI,Origin_PIR,Origin_FLG,Origin_CLT,Origin_BNA,Origin_PSC,Origin_BLI,Origin_ORH,Origin_ABQ,Origin_PBI,Origin_PIE,Origin_SDF,Origin_SCK,Origin_ACV,Origin_BDL,Origin_MRY,Origin_DAL,Origin_ITH,Origin_APF,Origin_CLE,Origin_TVL,Origin_PDX,Origin_MIA,Origin_MFR,Origin_ILG,Origin_TPA,Origin_BWI,Origin_ROA,Origin_OKC,Origin_SMF,Origin_YAP,Origin_MBS,Origin_ELM,Origin_PHX,Origin_FCA,Origin_STL,Origin_PWM,Origin_BET,Origin_DFW,Origin_ABE,Origin_MHT,Origin_GSP,Origin_LSE,Origin_STX,Origin_FAY,Origin_HDN,Origin_GUC,Origin_LMT,Origin_LBB,Origin_CRP,Origin_FSD,Origin_CSG,Origin_SFO,Origin_MEM,Origin_ELP,Origin_GCN,Origin_BHM,Origin_ATL,Origin_FLL,Origin_YKM,Origin_FNT,Origin_RIC,Origin_DAY,Origin_PHF,Origin_OMA,Origin_VPS,Origin_LIT,Origin_CAE,Origin_FAT,Origin_ICT,Origin_PFN,Origin_ORD,Origin_AVP,Origin_BTV,Origin_BIL,Origin_PUB,Origin_ILM,Origin_PIA,Origin_GUM,Origin_RDU,Origin_MFE,Origin_MKE,Origin_SYR,Origin_CCR,Origin_HSV,Origin_LFT,Origin_TUS,Origin_PIT,Origin_ACY,Origin_MDW,Origin_AZO,Origin_COS,Origin_OAJ,Origin_JNU,Origin_IND,Origin_ALO,Origin_KOA,Origin_EAU,Origin_GPT,Origin_MGM,Origin_HOU,Origin_DTW,Origin_TYS,Origin_CHA,Origin_YUM,Origin_MDT,Origin_ONT,Origin_FWA,Origin_JAX,Origin_LAX,Origin_MSP,Origin_HTS,Origin_MOT,Origin_BGR,Origin_BTR,Origin_SIT,Origin_MCO,Origin_ROC,Origin_ROR,Origin_SGF,Origin_AGS,Origin_OTZ,Origin_SAN,Origin_BZN,Origin_YAK,Origin_JFK,Origin_DAB,Origin_SUX,Origin_ANC,Origin_GNV,Origin_MSO,Origin_PHL,Origin_OGG,Origin_DSM,Origin_FOE,Origin_SAT,Origin_SLC,Origin_STT,Origin_RAP,Dest_BGM,Dest_MSY,Dest_GEG,Dest_BUR,Dest_SNA,Dest_GRB,Dest_GTF,Dest_IDA,Dest_GRR,Dest_EUG,Dest_PSG,Dest_PVD,Dest_GSO,Dest_MYR,Dest_ISO,Dest_OAK,Dest_FAR,Dest_BTM,Dest_MSN,Dest_SCC,Dest_DET,Dest_DCA,Dest_CID,Dest_MLU,Dest_WRG,Dest_HLN,Dest_LEX,Dest_RDM,Dest_ORF,Dest_EVV,Dest_KTN,Dest_CRW,Dest_CWA,Dest_SAV,Dest_TRI,Dest_CDV,Dest_CMH,Dest_SPN,Dest_CAK,Dest_

In [13]:
airline_subset = airlines.sample(False, 0.01)
#display(airline_subset.orderBy(F.rand()).take(100))

In [14]:
len(airline_subset)

In [15]:
# Catagorical: Year, Month, DayofMonth, DayofWeek, UniqueCarrier, FlightNumber, Origin, Dest
# Continous: Distance
# Input Time: CRSDepTime, CRSArrTime, CRSElapsedTime

his_data = airline_subset.toPandas()[['Distance', 'ArrDelay', 'DepDelay', 'ActualElapsedTime']]

# his_data.hist(figsize=(15,15), bins=15)
# display(plt.show())



/databricks/spark/python/pyspark/sql/types.py:1636: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 return pa.schema(fields)

In [16]:
for col in his_data.columns:
  plt.hist(his_data[col], log=True, bins=15)
  plt.xlabel(col)
  display(plt.show())
#Notes:
#Could be timestamp issues with super negative and positive delays, one day = 1440 minutes
#One sigle longest flight is about 18 hours from Sigapore to Newark
  
  # tempData = airline_subset.select('ActualElapsedTime') \
#         .withColumn("ActualElapsedTime", airlines["ActualElapsedTime"].cast(IntegerType())) \
#         .dropna() \
#         .rdd \
#         .flatMap(lambda x: x) \
#         .collect() 

# plt.figure(figsize = (15,15))
# plt.hist(tempData,bins=20)

# display(plt.show())

In [17]:
df = airline_subset.toPandas()
len(df)

Out[16]: 649564

In [18]:
df.to_csv('small_df.csv', index=False)
df.dropna(inplace = True)

In [19]:
#Approach:
#1) Drop bad data vs. fixable data
#2) Drop any NA arrival time and Elapsed time over 1100 and less than 10
#3) If depart time is way off, we want to consider to exclue the data, i.e. the depart time off 2 hours
#4) Create 2 models: 1 model is predicting departure time using arrival time, another is to predict actual laps time.  For final prediction we should use both models to predict arriv delay, but we train each model independently.  


# for i, col in enumerate(dumy.columns):
#   if dumy.iloc[0, i]!=0:
#     print(col, dumy[col].values)


# plt.scatter(df_pruned['Distance'], df_pruned['ActualElapsedTime'])
# display(plt.show())

# df_ontimDepart = df_pruned.query('-10<DepDelay<20')

# plt.scatter(df_ontimDepart['Distance'], df_ontimDepart['ActualElapsedTime'])
# display(plt.show())

df[df['ActualElapsedTime'].values.astype(int)<10]

,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,CRSElapsedTime,ArrDelay,DepDelay,Distance,IsArrDelayed,IsDepDelayed,Year_1987,Year_1988,Month_7,Month_11,Month_3,Month_8,Month_5,Month_6,Month_9,Month_1,Month_10,Month_4,Month_12,Month_2,DayofMonth_7,DayofMonth_15,DayofMonth_11,DayofMonth_29,DayofMonth_3,DayofMonth_30,DayofMonth_8,DayofMonth_22,DayofMonth_28,DayofMonth_16,DayofMonth_5,DayofMonth_31,DayofMonth_18,DayofMonth_27,...,Dest_MGM,Dest_HOU,Dest_DTW,Dest_TYS,Dest_CHA,Dest_YUM,Dest_ONT,Dest_MDT,Dest_FWA,Dest_JAX,Dest_LAX,Dest_MSP,Dest_MOT,Dest_HTS,Dest_BTR,Dest_BGR,Dest_SIT,Dest_MCO,Dest_ROC,Dest_SGF,Dest_AGS,Dest_OTZ,Dest_ROR,Dest_SAN,Dest_BZN,Dest_YAK,Dest_JFK,Dest_DAB,Dest_SUX,Dest_ANC,Dest_MSO,Dest_GNV,Dest_PHL,Dest_OGG,Dest_DSM,Dest_FOE,Dest_SAT,Dest_SLC,Dest_STT,Dest_RAP
13105,2221,2110,2228,2230,1897,7.0,80,-2.0,71.0,390.0,NO,YES,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
94162,130,30,217,120,1707,-13.0,50,57.0,60.0,261.0,YES,YES,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
104132,1827,1830,1834,1855,776,7.0,25,-21.0,-3.0,37.0,NO,NO,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
118589,1930,1930,1922,1930,547,-8.0,0,-8.0,0.0,185.0,NO,NO,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
157370,108,105,215,215,143,7.0,10,0.0,3.0,365.0,NO,YES,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
262230,700,600,800,805,705,0.0,65,-5.0,60.0,380.0,NO,YES,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
274367,1203,1150,1107,1155,856,4.0,65,-48.0,13.0,304.0,NO,YES,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
565691,1947,1845,1940,1930,288,-7.0,45,10.0,62.0,213.0,YES,YES,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
576434,2125,2125,2129,2155,3014,4.0,30,-26.0,0.0,24.0,NO,NO,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
def parse(line):
    """
    Map record_csv_string --> (tuple,of,fields)
    """
    fields = np.array(line, dtype = 'float')
    features,quality = fields[:-1], fields[-1]
    return(features, quality)

In [21]:

x_names = airline_subset.schema.names.copy()
x_names.remove('DepTime')
x_names.remove('FlightNum')
x_names.remove('ArrTime')
x_names.remove('ActualElapsedTime')
x_names.remove('ArrDelay')
x_names.remove('DepDelay')
x_names.remove('IsDepDelayed')
x_names.remove('IsArrDelayed')

# x_names = ['Distance','Year_1987', 'Year_1988','CRSDepTime','CRSArrTime','CRSElapsedTime']
clean_df = airline_subset.select(*x_names, 'ActualElapsedTime')
clean_df = clean_df.dropna()
# cache the training set 
xRDD = clean_df.rdd.map(parse).cache()

['DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'FlightNum', 'ActualElapsedTime', 'CRSElapsedTime', 'ArrDelay', 'DepDelay', 'Distance', 'IsArrDelayed', 'IsDepDelayed', 'Year_1987', 'Year_1988', 'Month_7', 'Month_11', 'Month_3', 'Month_8', 'Month_5', 'Month_6', 'Month_9', 'Month_1', 'Month_10', 'Month_4', 'Month_12', 'Month_2', 'DayofMonth_7', 'DayofMonth_15', 'DayofMonth_11', 'DayofMonth_29', 'DayofMonth_3', 'DayofMonth_30', 'DayofMonth_8', 'DayofMonth_22', 'DayofMonth_28', 'DayofMonth_16', 'DayofMonth_5', 'DayofMonth_31', 'DayofMonth_18', 'DayofMonth_27', 'DayofMonth_17', 'DayofMonth_26', 'DayofMonth_6', 'DayofMonth_19', 'DayofMonth_23', 'DayofMonth_25', 'DayofMonth_24', 'DayofMonth_9', 'DayofMonth_1', 'DayofMonth_20', 'DayofMonth_10', 'DayofMonth_4', 'DayofMonth_12', 'DayofMonth_13', 'DayofMonth_21', 'DayofMonth_14', 'DayofMonth_2', 'DayOfWeek_7', 'DayOfWeek_3', 'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_1', 'DayOfWeek_4', 'DayOfWeek_2', 'UniqueCarrier_UA', 'UniqueCarrier_EA', 'UniqueCarrier_PI', 'UniqueCarrier_PS', 'UniqueCarrier_AA', 'UniqueCarrier_NW', 'UniqueCarrier_HP', 'UniqueCarrier_TW', 'UniqueCarrier_DL', 'UniqueCarrier_US', 'UniqueCarrier_AS', 'UniqueCarrier_CO', 'UniqueCarrier_PA (1)', 'UniqueCarrier_WN', 'Origin_BGM', 'Origin_MSY', 'Origin_GEG', 'Origin_BUR', 'Origin_SNA', 'Origin_GTF', 'Origin_GRB', 'Origin_IDA', 'Origin_GRR', 'Origin_EUG', 'Origin_PSG', 'Origin_MYR', 'Origin_GSO', 'Origin_PVD', 'Origin_ISO', 'Origin_OAK', 'Origin_MSN', 'Origin_FAR', 'Origin_BTM', 'Origin_SCC', 'Origin_DET', 'Origin_DCA', 'Origin_CID', 'Origin_MLU', 'Origin_WRG', 'Origin_HLN', 'Origin_LEX', 'Origin_RDM', 'Origin_ORF', 'Origin_EVV', 'Origin_KTN', 'Origin_CRW', 'Origin_CWA', 'Origin_SAV', 'Origin_TRI', 'Origin_CDV', 'Origin_CMH', 'Origin_SPN', 'Origin_UCA', 'Origin_CAK', 'Origin_CHO', 'Origin_MOB', 'Origin_PNS', 'Origin_LIH', 'Origin_IAH', 'Origin_HNL', 'Origin_ERI', 'Origin_SHV', 'Origin_GST', 'Origin_CVG', 'Origin_SJC', 'Origin_TOL', 'Origin_LGA', 'Origin_BUF', 'Origin_TLH', 'Origin_HPN', 'Origin_RDD', 'Origin_AUS', 'Origin_MLI', 'Origin_SJU', 'Origin_ATW', 'Origin_LYH', 'Origin_AVL', 'Origin_GJT', 'Origin_LGB', 'Origin_BFL', 'Origin_GFK', 'Origin_RNO', 'Origin_SRQ', 'Origin_EYW', 'Origin_SBN', 'Origin_JAC', 'Origin_RST', 'Origin_CHS', 'Origin_RSW', 'Origin_TUL', 'Origin_HRL', 'Origin_AMA', 'Origin_ISP', 'Origin_BOS', 'Origin_MAF', 'Origin_MLB', 'Origin_EWR', 'Origin_LAS', 'Origin_BIS', 'Origin_FAI', 'Origin_JAN', 'Origin_DLH', 'Origin_DEN', 'Origin_ALB', 'Origin_CPR', 'Origin_LNK', 'Origin_OME', 'Origin_IAD', 'Origin_PSP', 'Origin_SBA', 'Origin_BOI', 'Origin_DRO', 'Origin_SEA', 'Origin_CMI', 'Origin_LAN', 'Origin_MCI', 'Origin_PIR', 'Origin_FLG', 'Origin_CLT', 'Origin_BNA', 'Origin_PSC', 'Origin_BLI', 'Origin_ORH', 'Origin_ABQ', 'Origin_PBI', 'Origin_PIE', 'Origin_SDF', 'Origin_SCK', 'Origin_ACV', 'Origin_BDL', 'Origin_MRY', 'Origin_DAL', 'Origin_ITH', 'Origin_APF', 'Origin_CLE', 'Origin_TVL', 'Origin_PDX', 'Origin_MIA', 'Origin_MFR', 'Origin_ILG', 'Origin_TPA', 'Origin_BWI', 'Origin_ROA', 'Origin_OKC', 'Origin_SMF', 'Origin_YAP', 'Origin_MBS', 'Origin_ELM', 'Origin_PHX', 'Origin_FCA', 'Origin_STL', 'Origin_PWM', 'Origin_BET', 'Origin_DFW', 'Origin_ABE', 'Origin_MHT', 'Origin_GSP', 'Origin_LSE', 'Origin_STX', 'Origin_FAY', 'Origin_HDN', 'Origin_GUC', 'Origin_LMT', 'Origin_LBB', 'Origin_CRP', 'Origin_FSD', 'Origin_CSG', 'Origin_SFO', 'Origin_MEM', 'Origin_ELP', 'Origin_GCN', 'Origin_BHM', 'Origin_ATL', 'Origin_FLL', 'Origin_YKM', 'Origin_FNT', 'Origin_RIC', 'Origin_DAY', 'Origin_PHF', 'Origin_OMA', 'Origin_VPS', 'Origin_LIT', 'Origin_CAE', 'Origin_FAT', 'Origin_ICT', 'Origin_PFN', 'Origin_ORD', 'Origin_AVP', 'Origin_BTV', 'Origin_BIL', 'Origin_PUB', 'Origin_ILM', 'Origin_PIA', 'Origin_GUM', 'Origin_RDU', 'Origin_MFE', 'Origin_MKE', 'Origin_SYR', 'Origin_CCR', 'Origin_HSV', 'Origin_LFT', 'Origin_TUS', 'Origin_PIT', 'Origin_ACY', 'Origin_MDW', 'Origin_AZO', 'Origin_COS', 'Origin_OAJ', 'Origin_JNU', 'Origin_IND', 'Origin_ALO', 'Or

In [22]:
print(x_names)


['CRSDepTime', 'CRSArrTime', 'CRSElapsedTime', 'Distance', 'Year_1987', 'Year_1988', 'Month_7', 'Month_11', 'Month_3', 'Month_8', 'Month_5', 'Month_6', 'Month_9', 'Month_1', 'Month_10', 'Month_4', 'Month_12', 'Month_2', 'DayofMonth_7', 'DayofMonth_15', 'DayofMonth_11', 'DayofMonth_29', 'DayofMonth_3', 'DayofMonth_30', 'DayofMonth_8', 'DayofMonth_22', 'DayofMonth_28', 'DayofMonth_16', 'DayofMonth_5', 'DayofMonth_31', 'DayofMonth_18', 'DayofMonth_27', 'DayofMonth_17', 'DayofMonth_26', 'DayofMonth_6', 'DayofMonth_19', 'DayofMonth_23', 'DayofMonth_25', 'DayofMonth_24', 'DayofMonth_9', 'DayofMonth_1', 'DayofMonth_20', 'DayofMonth_10', 'DayofMonth_4', 'DayofMonth_12', 'DayofMonth_13', 'DayofMonth_21', 'DayofMonth_14', 'DayofMonth_2', 'DayOfWeek_7', 'DayOfWeek_3', 'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_1', 'DayOfWeek_4', 'DayOfWeek_2', 'UniqueCarrier_UA', 'UniqueCarrier_EA', 'UniqueCarrier_PI', 'UniqueCarrier_PS', 'UniqueCarrier_AA', 'UniqueCarrier_NW', 'UniqueCarrier_HP', 'UniqueCarrier_TW', 'UniqueCarrier_DL', 'UniqueCarrier_US', 'UniqueCarrier_AS', 'UniqueCarrier_CO', 'UniqueCarrier_PA (1)', 'UniqueCarrier_WN', 'Origin_BGM', 'Origin_MSY', 'Origin_GEG', 'Origin_BUR', 'Origin_SNA', 'Origin_GTF', 'Origin_GRB', 'Origin_IDA', 'Origin_GRR', 'Origin_EUG', 'Origin_PSG', 'Origin_MYR', 'Origin_GSO', 'Origin_PVD', 'Origin_ISO', 'Origin_OAK', 'Origin_MSN', 'Origin_FAR', 'Origin_BTM', 'Origin_SCC', 'Origin_DET', 'Origin_DCA', 'Origin_CID', 'Origin_MLU', 'Origin_WRG', 'Origin_HLN', 'Origin_LEX', 'Origin_RDM', 'Origin_ORF', 'Origin_EVV', 'Origin_KTN', 'Origin_CRW', 'Origin_CWA', 'Origin_SAV', 'Origin_TRI', 'Origin_CDV', 'Origin_CMH', 'Origin_SPN', 'Origin_UCA', 'Origin_CAK', 'Origin_CHO', 'Origin_MOB', 'Origin_PNS', 'Origin_LIH', 'Origin_IAH', 'Origin_HNL', 'Origin_ERI', 'Origin_SHV', 'Origin_GST', 'Origin_CVG', 'Origin_SJC', 'Origin_TOL', 'Origin_LGA', 'Origin_BUF', 'Origin_TLH', 'Origin_HPN', 'Origin_RDD', 'Origin_AUS', 'Origin_MLI', 'Origin_SJU', 'Origin_ATW', 'Origin_LYH', 'Origin_AVL', 'Origin_GJT', 'Origin_LGB', 'Origin_BFL', 'Origin_GFK', 'Origin_RNO', 'Origin_SRQ', 'Origin_EYW', 'Origin_SBN', 'Origin_JAC', 'Origin_RST', 'Origin_CHS', 'Origin_RSW', 'Origin_TUL', 'Origin_HRL', 'Origin_AMA', 'Origin_ISP', 'Origin_BOS', 'Origin_MAF', 'Origin_MLB', 'Origin_EWR', 'Origin_LAS', 'Origin_BIS', 'Origin_FAI', 'Origin_JAN', 'Origin_DLH', 'Origin_DEN', 'Origin_ALB', 'Origin_CPR', 'Origin_LNK', 'Origin_OME', 'Origin_IAD', 'Origin_PSP', 'Origin_SBA', 'Origin_BOI', 'Origin_DRO', 'Origin_SEA', 'Origin_CMI', 'Origin_LAN', 'Origin_MCI', 'Origin_PIR', 'Origin_FLG', 'Origin_CLT', 'Origin_BNA', 'Origin_PSC', 'Origin_BLI', 'Origin_ORH', 'Origin_ABQ', 'Origin_PBI', 'Origin_PIE', 'Origin_SDF', 'Origin_SCK', 'Origin_ACV', 'Origin_BDL', 'Origin_MRY', 'Origin_DAL', 'Origin_ITH', 'Origin_APF', 'Origin_CLE', 'Origin_TVL', 'Origin_PDX', 'Origin_MIA', 'Origin_MFR', 'Origin_ILG', 'Origin_TPA', 'Origin_BWI', 'Origin_ROA', 'Origin_OKC', 'Origin_SMF', 'Origin_YAP', 'Origin_MBS', 'Origin_ELM', 'Origin_PHX', 'Origin_FCA', 'Origin_STL', 'Origin_PWM', 'Origin_BET', 'Origin_DFW', 'Origin_ABE', 'Origin_MHT', 'Origin_GSP', 'Origin_LSE', 'Origin_STX', 'Origin_FAY', 'Origin_HDN', 'Origin_GUC', 'Origin_LMT', 'Origin_LBB', 'Origin_CRP', 'Origin_FSD', 'Origin_CSG', 'Origin_SFO', 'Origin_MEM', 'Origin_ELP', 'Origin_GCN', 'Origin_BHM', 'Origin_ATL', 'Origin_FLL', 'Origin_YKM', 'Origin_FNT', 'Origin_RIC', 'Origin_DAY', 'Origin_PHF', 'Origin_OMA', 'Origin_VPS', 'Origin_LIT', 'Origin_CAE', 'Origin_FAT', 'Origin_ICT', 'Origin_PFN', 'Origin_ORD', 'Origin_AVP', 'Origin_BTV', 'Origin_BIL', 'Origin_PUB', 'Origin_ILM', 'Origin_PIA', 'Origin_GUM', 'Origin_RDU', 'Origin_MFE', 'Origin_MKE', 'Origin_SYR', 'Origin_CCR', 'Origin_HSV', 'Origin_LFT', 'Origin_TUS', 'Origin_PIT', 'Origin_ACY', 'Origin_MDW', 'Origin_AZO', 'Origin_COS', 'Origin_OAJ', 'Origin_JNU', 'Origin_IND', 'Origin_ALO', 'Origin_KOA', 'Origin_EAU', 'Origin_GPT', 'Origin_MGM', 'Origin_HOU', 'Origin_DTW', 'Origin_TYS', 'Origin_CHA', 'Or

In [23]:
def OLSLoss(dataRDD, W):
    """
    Compute mean squared error.
    Args:
        dataRDD - each record is a tuple of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    """
    
    
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1]))
  
    loss = augmentedData.map(lambda x: float(np.square(np.dot(x[0],W) - x[1]))).sum() / augmentedData.count()
    
 
    return loss

In [24]:
w = [0]*(len(x_names)+1)
w[x_names.index('CRSElapsedTime')+1] = 1
OLSLoss(xRDD, w)

Out[65]: 139.2643241537077